In [16]:
pip install transformers datasets



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install sentencepiece


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install torch



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [19]:
import sys
print(sys.version)
print(sys.executable)

3.11.10 (main, Sep  7 2024, 01:03:31) [Clang 15.0.0 (clang-1500.3.9.4)]
/opt/homebrew/opt/python@3.11/bin/python3.11


In [20]:
pip install transformers

^C
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/homebrew/lib/python3.11/site-packages/pip/__main__.py", line 22, in <module>
    from pip._internal.cli.main import main as _main
  File "/opt/homebrew/lib/python3.11/site-packages/pip/_internal/cli/main.py", line 11, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/opt/homebrew/lib/python3.11/site-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/opt/homebrew/lib/python3.11/site-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/opt/homebrew/lib/python3.11/site-packages/pip/_internal/build_env.py", line 18, in <module>
    from pip._internal.cli.spinners import open_spinner
  File "/opt/homebrew/lib/python3.11/site-packages/pip/_i

In [ ]:

# Step 2: Import necessary libraries
from transformers import BartForConditionalGeneration, BartTokenizer, T5ForConditionalGeneration, T5Tokenizer
from datasets import load_dataset


# Step 3: Load the CNN/DailyMail dataset
dataset = load_dataset("cnn_dailymail")


/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

# Step 4: Load BART and T5 models and tokenizers
bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

t5_model = T5ForConditionalGeneration.from_pretrained("t5-large")
t5_tokenizer = T5Tokenizer.from_pretrained("t5-large")



You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
#  Step 5: Define summarization functions for each model

# Function to summarize with BART
def summarize_bart(text, max_length=130, min_length=30):
    inputs = bart_tokenizer([text], max_length=1024, return_tensors="pt", truncation=True)
    summary_ids = bart_model.generate(inputs["input_ids"], max_length=max_length, min_length=min_length, length_penalty=2.0, num_beams=4, early_stopping=True)
    return bart_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Function to summarize with T5
def summarize_t5(text, max_length=130, min_length=30):
    inputs = t5_tokenizer("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = t5_model.generate(inputs["input_ids"], max_length=max_length, min_length=min_length, length_penalty=2.0, num_beams=4, early_stopping=True)
    return t5_tokenizer.decode(summary_ids[0], skip_special_tokens=True)


In [ ]:
pip install rouge-score



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from rouge_score import rouge_scorer

# Function to calculate ROUGE scores
def calculate_rouge_scores(reference, hypothesis):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, hypothesis)
    return scores

# Function to calculate word overlap accuracy
def calculate_accuracy(reference, hypothesis):
    reference_words = set(reference.split())
    hypothesis_words = set(hypothesis.split())
    common_words = reference_words.intersection(hypothesis_words)
    accuracy = len(common_words) / len(reference_words) if len(reference_words) > 0 else 0
    return accuracy

In [ ]:

# Step 6: Test summarization on a sample from the validation set
sample = dataset['validation'][0]
article = sample['article']
reference_summary = sample['highlights']

# Generate summaries
bart_summary = summarize_bart(article)
t5_summary = summarize_t5(article)

# Calculate metrics for BART
bart_rouge_scores = calculate_rouge_scores(reference_summary, bart_summary)
bart_accuracy = calculate_accuracy(reference_summary, bart_summary)

# Calculate metrics for T5
t5_rouge_scores = calculate_rouge_scores(reference_summary, t5_summary)
t5_accuracy = calculate_accuracy(reference_summary, t5_summary)




In [ ]:
# Display results
print("Original Summary:\n", reference_summary)
print("\nBART Summary:\n", bart_summary)
print("\nBART ROUGE Scores:")
for metric, score in bart_rouge_scores.items():
    print(f"{metric}: Precision={score.precision:.4f}, Recall={score.recall:.4f}, F1={score.fmeasure:.4f}")
print(f"\nBART Word Overlap Accuracy: {bart_accuracy:.2%}")

print("\nT5 Summary:\n", t5_summary)
print("\nT5 ROUGE Scores:")
for metric, score in t5_rouge_scores.items():
    print(f"{metric}: Precision={score.precision:.4f}, Recall={score.recall:.4f}, F1={score.fmeasure:.4f}")
print(f"\nT5 Word Overlap Accuracy: {t5_accuracy:.2%}")

Original Summary:
 Sally Forrest, an actress-dancer who graced the silver screen throughout the '40s and '50s in MGM musicals and films died on March 15 .
Forrest, whose birth name was Katherine Feeney, had long battled cancer .
A San Diego native, Forrest became a protege of Hollywood trailblazer Ida Lupino, who cast her in starring roles in films .

BART Summary:
 Forrest, whose birth name was Katherine Feeney, was 86 and had long battled cancer. A San Diego native, Forrest became a protege of Hollywood trailblazer Ida Lupino, who cast her in starring roles in films including Not Wanted, Never Fear and Hard, Fast and Beautiful. Forrest appeared as herself in an episode of The Ed Sullivan Show and three episodes of The Dinah Shore Chevy Show.

BART ROUGE Scores:
rouge1: Precision=0.5909, Recall=0.6964, F1=0.6393
rouge2: Precision=0.4615, Recall=0.5455, F1=0.5000
rougeL: Precision=0.4848, Recall=0.5714, F1=0.5246

BART Word Overlap Accuracy: 65.31%

T5 Summary:
 forrest, whose birth na

In [ ]:
# fine-tuned BART model - Load CNN/DailyMail dataset
dataset = load_dataset("cnn_dailymail")
train_data = dataset['train']
val_data = dataset['validation']

# Initialize BART tokenizer
bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

# Tokenize function for preparing data
def tokenize_data(example):
    inputs = bart_tokenizer(example["article"], max_length=1024, truncation=True, padding="max_length", return_tensors="pt")
    outputs = bart_tokenizer(example["highlights"], max_length=128, truncation=True, padding="max_length", return_tensors="pt")
    example["input_ids"] = inputs.input_ids.squeeze()
    example["attention_mask"] = inputs.attention_mask.squeeze()
    example["labels"] = outputs.input_ids.squeeze()
    return example

# Tokenize train and validation data
train_data = train_data.map(tokenize_data, batched=True)
val_data = val_data.map(tokenize_data, batched=True)


In [ ]:
pip install "transformers[torch]"


  Using cached accelerate-1.1.1-py3-none-any.whl.metadata (19 kB)
Using cached accelerate-1.1.1-py3-none-any.whl (333 kB)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip uninstall -y accelerate

Found existing installation: accelerate 1.1.1
Uninstalling accelerate-1.1.1:
  Successfully uninstalled accelerate-1.1.1
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install "transformers[torch,accelerate]"


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import accelerate

accelerate.__version__

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'1.1.1'

In [67]:
import torch
import logging
import random
from datasets import load_dataset
from transformers import BartForConditionalGeneration, BartTokenizer, DataCollatorForSeq2Seq, Trainer, TrainingArguments

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Device selection
device = torch.device("cpu")
logger.info(f"Using device: {device}")

# Load the CNN/DailyMail dataset
logger.info("Loading CNN/DailyMail dataset...")
dataset = load_dataset("cnn_dailymail")

# Increase the dataset size
train_size = 5000  # Adjust this number based on your system's capacity
val_size = 500
logger.info(f"Selecting {train_size} samples for training and {val_size} samples for validation.")

# Shuffle and select data for diversity
train_indices = random.sample(range(len(dataset['train'])), train_size)
val_indices = random.sample(range(len(dataset['validation'])), val_size)
train_data = dataset['train'].select(train_indices)
val_data = dataset['validation'].select(val_indices)

# Initialize the model and tokenizer
model_name = "sshleifer/distilbart-cnn-12-6"  # A slightly larger model for better performance
logger.info(f"Loading model and tokenizer: {model_name}")
bart_model = BartForConditionalGeneration.from_pretrained(model_name).to(device)
bart_tokenizer = BartTokenizer.from_pretrained(model_name)

# Tokenize function
def tokenize_data(example):
    inputs = bart_tokenizer(
        example["article"],
        max_length=512,
        truncation=True,
        padding='max_length'
    )
    outputs = bart_tokenizer(
        example["highlights"],
        max_length=128,
        truncation=True,
        padding='max_length'
    )
    example["input_ids"] = inputs["input_ids"]
    example["attention_mask"] = inputs["attention_mask"]
    example["labels"] = outputs["input_ids"]
    return example

# Tokenize the data
logger.info("Tokenizing training data...")
train_data = train_data.map(
    tokenize_data,
    batched=True,
    remove_columns=["article", "highlights", "id"]
)
logger.info("Tokenizing validation data...")
val_data = val_data.map(
    tokenize_data,
    batched=True,
    remove_columns=["article", "highlights", "id"]
)

# Set the format for PyTorch
logger.info("Setting dataset format for PyTorch...")
train_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
val_data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Initialize data collator
logger.info("Initializing data collator...")
data_collator = DataCollatorForSeq2Seq(
    tokenizer=bart_tokenizer,
    model=bart_model,
    padding=True,
)

# Define training arguments
logger.info("Setting up training arguments...")
training_args = TrainingArguments(
    output_dir="./custom_bart_model",
    evaluation_strategy="epoch",
    logging_strategy="steps",
    logging_steps=500,
    learning_rate=5e-5,  # Slightly higher learning rate
    per_device_train_batch_size=1,  # Keep batch size small for memory management
    per_device_eval_batch_size=1,
    num_train_epochs=3,  # Increased number of epochs
    weight_decay=0.01,
    save_total_limit=2,
    save_steps=1000,
)

# Initialize Trainer
logger.info("Initializing Trainer...")
trainer = Trainer(
    model=bart_model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    data_collator=data_collator,
    tokenizer=bart_tokenizer,
)

# Train the model
logger.info("Starting training...")
trainer.train()
logger.info("Training completed.")

# Save the model and tokenizer
logger.info("Saving the trained model and tokenizer...")
bart_model.save_pretrained('./custom_bart_model_pretrained')
bart_tokenizer.save_pretrained('./custom_bart_model_pretrained')
logger.info("Model and tokenizer saved successfully using save_pretrained.")


Map: 100%|██████████| 500/500 [00:01<00:00, 408.97 examples/s]
/opt/homebrew/lib/python3.11/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/folders/47/fhfgvqzj5739ymb6rclbq1j40000gn/T/ipykernel_44929/3923815360.py:100: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
  3%|▎         | 500/15000 [03:50<1:49:41,  2.20it/s]

{'loss': 1.2511, 'grad_norm': 6.305051803588867, 'learning_rate': 4.8333333333333334e-05, 'epoch': 0.1}


  7%|▋         | 1000/15000 [07:39<1:46:54,  2.18it/s]/opt/homebrew/lib/python3.11/site-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


{'loss': 1.0999, 'grad_norm': 8.382759094238281, 'learning_rate': 4.666666666666667e-05, 'epoch': 0.2}


 10%|█         | 1500/15000 [11:31<1:41:53,  2.21it/s]

...

{'loss': 0.3735, 'grad_norm': 7.689404487609863, 'learning_rate': 1.6666666666666667e-06, 'epoch': 2.9}


100%|██████████| 15000/15000 [1:59:20<00:00,  2.19it/s]

{'loss': 0.3573, 'grad_norm': 4.589293479919434, 'learning_rate': 0.0, 'epoch': 3.0}



100%|██████████| 15000/15000 [2:00:02<00:00,  2.08it/s]


{'eval_loss': 1.3798279762268066, 'eval_runtime': 37.3695, 'eval_samples_per_second': 13.38, 'eval_steps_per_second': 13.38, 'epoch': 3.0}
{'train_runtime': 7202.0447, 'train_samples_per_second': 2.083, 'train_steps_per_second': 2.083, 'train_loss': 0.7272747873942057, 'epoch': 3.0}


In [69]:
pip install evaluate


Python(61379) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [75]:
pip install sacrebleu

Python(61430) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 10.3 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [78]:
import pandas as pd
import nltk
from datasets import load_dataset
import evaluate
from transformers import BartForConditionalGeneration, BartTokenizer, T5ForConditionalGeneration, T5Tokenizer
from rouge_score import rouge_scorer

# Ensure nltk is downloaded for BLEU scoring
nltk.download('punkt')

# Load BLEU and ROUGE metrics
bleu = evaluate.load("sacrebleu")
rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Step 1: Load the cnn_dailymail dataset and save it to CSV files
dataset = load_dataset("cnn_dailymail")

# Convert and save each split as CSV
train_df = pd.DataFrame(dataset['train'])
train_df.to_csv("cnn_dailymail_train.csv", index=False)

validation_df = pd.DataFrame(dataset['validation'])
validation_df.to_csv("cnn_dailymail_validation.csv", index=False)

test_df = pd.DataFrame(dataset['test'])
test_df.to_csv("cnn_dailymail_test.csv", index=False)

print("Saved train, validation, and test splits as CSV files.")

# Step 2: Load models and tokenizers
custom_bart_model = BartForConditionalGeneration.from_pretrained('./custom_bart_model_pretrained')
bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
t5_model = T5ForConditionalGeneration.from_pretrained("t5-large")

bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
t5_tokenizer = T5Tokenizer.from_pretrained("t5-large")

# Function to evaluate and compare summaries
def evaluate_summary(model, tokenizer, text, reference_summary, max_length=130, min_length=30):
    inputs = tokenizer(text, max_length=1024, return_tensors="pt", truncation=True)
    summary_ids = model.generate(inputs["input_ids"], max_length=max_length, min_length=min_length, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    # Calculate ROUGE scores
    rouge_scores = rouge.score(reference_summary, summary)
    
    # Calculate BLEU score
    bleu_score = bleu.compute(predictions=[summary], references=[[reference_summary]])
    
    return summary, rouge_scores, bleu_score

# Step 3: Function to evaluate on a single row in the CSV dataset
def evaluate_single_row(file_path):
    df = pd.read_csv(file_path)

    # Select the first row for testing
    row = df.iloc[0]
    article = row['article']  # Adjust if column names differ
    reference_summary = row['highlights']  # Adjust if column names differ
    
    # Evaluate custom BART model
    custom_bart_summary, custom_bart_rouge, custom_bart_bleu = evaluate_summary(custom_bart_model, bart_tokenizer, article, reference_summary)
    print("\nCustom BART Model Summary:", custom_bart_summary)
    print("Custom BART ROUGE:", custom_bart_rouge)
    print("Custom BART BLEU:", custom_bart_bleu['score'])
    
    # Evaluate pre-trained BART model
    pretrained_bart_summary, pretrained_bart_rouge, pretrained_bart_bleu = evaluate_summary(bart_model, bart_tokenizer, article, reference_summary)
    print("\nPre-trained BART Model Summary:", pretrained_bart_summary)
    print("Pre-trained BART ROUGE:", pretrained_bart_rouge)
    print("Pre-trained BART BLEU:", pretrained_bart_bleu['score'])
    
    # Evaluate T5 model
    t5_summary, t5_rouge, t5_bleu = evaluate_summary(t5_model, t5_tokenizer, article, reference_summary)
    print("\nT5 Model Summary:", t5_summary)
    print("T5 ROUGE:", t5_rouge)
    print("T5 BLEU:", t5_bleu['score'])

# Step 4: Run the test on a single row from the validation CSV
evaluate_single_row("cnn_dailymail_validation.csv")


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/darshankarkera/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Saved train, validation, and test splits as CSV files.


/opt/homebrew/lib/python3.11/site-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/transformers/generation/utils.py:1493: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(



Custom BART Model Summary: Sally Forrest, 86, was a protege of Hollywood trailblazer Ida Lupino, who cast her in starring roles in films such as While the City Sleeps and The Seven Year Itch .
Forrest, whose birth name was Katherine Feeney, was a dancer and actress .
She is survived by her niece, Sharon Durham, and nephews, Michael and Mark Feeney .
Custom BART ROUGE: {'rouge1': Score(precision=0.5535714285714286, recall=0.5535714285714286, fmeasure=0.5535714285714286), 'rouge2': Score(precision=0.38181818181818183, recall=0.38181818181818183, fmeasure=0.38181818181818183), 'rougeL': Score(precision=0.32142857142857145, recall=0.32142857142857145, fmeasure=0.32142857142857145)}
Custom BART BLEU: 38.83933530357128

Pre-trained BART Model Summary: Forrest, whose birth name was Katherine Feeney, was 86 and had long battled cancer. A San Diego native, Forrest became a protege of Hollywood trailblazer Ida Lupino, who cast her in starring roles in films including Not Wanted, Never Fear and 